In [1]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src.models import ModelXtoCResNet
from src.preprocessing.Derm7pt import preprocessing_Derm7pt
from src.utils import *
from src.training import run_epoch_x_to_c

In [2]:
concept_labels, train_loader, val_loader, test_loader = preprocessing_Derm7pt(training=True, class_concepts=True, verbose=True)

Total number of label columns: 5
Found 1011 instances.
Created matrix of shape: (1011, 5)
Total number of concept columns: 19
Found 2013 images.
Processing in 32 batches of size 64 (for progress reporting)...


Processing batches: 100%|██████████| 32/32 [00:15<00:00,  2.04it/s]


Finished processing.
Successfully transformed: 2013 images.
Labels shape: (2013, 5)
Concepts shape: (2013, 19)
Image tensors length: 2013
Dataset initialized with 413 pre-sorted items.
Dataset initialized with 203 pre-sorted items.
Dataset initialized with 395 pre-sorted items.


# Training Implementation

In [3]:
from src.utils import find_class_imbalance
from src.config import DERM7PT_CONFIG as config_dict
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
N_TRIMMED_CONCEPTS = config_dict['N_TRIMMED_CONCEPTS']
N_CLASSES = config_dict['N_CLASSES']

**Find device to run model on (CPU or GPU).**

In [5]:
device = torch.device("cuda" if torch.cuda.is_available()
                    else "mps" if torch.backends.mps.is_available()
                    else "cpu")
print(f"Using device: {device}")

Using device: mps


**Instantiate the model.**

In [6]:
model = ModelXtoCResNet(pretrained=True,
                freeze=True,
                n_concepts=N_TRIMMED_CONCEPTS)

model = model.to(device)
print("Model Instantiated (X -> C)")

/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Instantiated (X -> C)


### Loss
We use weighted loss.

`BCEWithLogitsLoss()` performs 2 steps:
1. $\sigma(x)$
    - Applies the sigmoid function to the logits to get probabilities.
2. $\text{BCE}(\sigma(x), y) = y \cdot \text{log}(\sigma(x)) + (1-y) \cdot (1-\text{log}(\sigma(x)))$
    - Compute binary cross-entropy between output probabilities ($\sigma(x)$) and ground truths ($y$)

In [7]:
use_weighted_loss = True # Set to False for simple unweighted loss

if use_weighted_loss:
    concept_weights = find_class_imbalance(concept_labels)
    attr_criterion = [nn.BCEWithLogitsLoss(weight=torch.tensor([ratio], device=device, dtype=torch.float))
                    for ratio in concept_weights]
else:
    attr_criterion = [nn.BCEWithLogitsLoss() for _ in range(N_TRIMMED_CONCEPTS)]

### Optimiser
Use same settings as used in CBM repo.

In [8]:
lr = 0.01
weight_decay = 0.00004 # same as lambda in L2-regularisation

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                    lr=lr,
                    momentum=0.9,
                    weight_decay=weight_decay)

# scheduler_step = n -> decrease the LR every n epochs
scheduler_step = 1000
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=0.1)

print("Optimizer and Scheduler Ready")

Optimizer and Scheduler Ready


### Training and Validation Loops

In [ ]:
epochs = 50
log_interval = 50

best_val_acc = 0.0

: 

In [ ]:
for epoch in range(epochs):
    print(f"--- Epoch {epoch+1}/{epochs} ---")

    # Train
    train_loss, train_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, is_training=True, use_aux=True, n_concepts=N_TRIMMED_CONCEPTS, device=device, verbose=True)
    print(f'Epoch {epoch+1} Train Summary | Loss: {train_loss:.4f} | Acc: {train_acc:.3f}')

    # Validate
    if val_loader:
        with torch.no_grad():
            val_loss, val_acc = run_epoch_x_to_c(model, val_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device, verbose=True)

        print(f'Epoch {epoch+1} Val Summary   | Loss: {val_loss:.4f} | Acc: {val_acc:.3f}')

        # Save best model based on validation accuracy
        if val_acc > best_val_acc:
            print(f"Validation accuracy improved ({best_val_acc:.3f} -> {val_acc:.3f}). Saving model...")
            best_val_acc = val_acc
            torch.save(model, 'x_to_c_best_model.pth')
            print("Model saved to x_to_c_best_model.pth")

    # Scheduler step
    scheduler.step()
    print(f"Current LR: {optimizer.param_groups[0]['lr']}")

--- Epoch 1/50 ---


Epoch 1 Train Summary | Loss: 2.5328 | Acc: 78.387


Epoch 1 Val Summary   | Loss: 2.0438 | Acc: 82.629
Validation accuracy improved (0.000 -> 82.629). Saving model...
Model saved to x_to_c_best_model.pth
Current LR: 0.01
--- Epoch 2/50 ---


Epoch 2 Train Summary | Loss: 1.8151 | Acc: 87.091


Epoch 2 Val Summary   | Loss: 1.7277 | Acc: 83.796
Validation accuracy improved (82.629 -> 83.796). Saving model...
Model saved to x_to_c_best_model.pth
Current LR: 0.01
--- Epoch 3/50 ---


...

Epoch 7 Train Summary | Loss: 1.1546 | Acc: 89.525


Epoch 7 Val Summary   | Loss: 1.4860 | Acc: 84.936
Current LR: 0.01
--- Epoch 8/50 ---


Epoch 8 Train Summary | Loss: 1.1070 | Acc: 89.945


Epoch 8 Val Summary   | Loss: 1.4488 | Acc: 85.844
Validation accuracy improved (85.118 -> 85.844). Saving model...
Model saved to x_to_c_best_model.pth
Current LR: 0.01
--- Epoch 9/50 ---


Training:  14%|█▍        | 1/7 [00:02<00:13,  2.29s/it, acc=90.7072, loss=1.0624]libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
libc++abi: terminating due to uncaught exception of type std::__1::system_error: Broken pipe
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x13245a8e0>
Traceback (most recent call last):
  File "/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torch/utils/data/dataloader.py", line 1582, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/multiprocessing/process.py",

In [ ]:
if test_loader:
    with torch.no_grad():

        test_loss, test_acc, outputs = run_epoch_x_to_c(
            model, test_loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device,
            return_outputs='sigmoid', verbose=True
        )

# print(f"Shuffled labels shape: {shuffled_img_labels.shape}")
print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')